In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
model = load_model('mymodel.keras')
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [15]:
#Example input data
input_data = {
    'CreditScore': 600,
    'Geography' : 'France',
    'Gender': 'Male',
    'Age' : 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [19]:
input_df = pd.DataFrame([input_data])

In [10]:
geo_encoded = ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_endoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_endoded_df


e:\ITWork\AI_Langchain\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [21]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [22]:
## Concatination with OHE data
input_data_0 = pd.concat([input_df.drop('Geography', axis=1), geo_endoded_df], axis=1)
input_data_0

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [24]:
input_scaled = scalar.transform(input_data_0)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [25]:
prediction = model.predict(input_scaled)
pred_prob = prediction[0][0]
if pred_prob > 0.5:
    print ("The customer is likely to be churn")
else:
    print("The customer is not likely to be churn")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
The customer is not likely to be churn
